In [2]:
import json

In [3]:
with open("../../data/all_qa_cleaned.json") as file:
    intents = json.load(file)

In [10]:
prompt_template = """You emulate a user of an assistant that answers computer science or data science questions.
Formulate 5 alternative questions that the user may ask, based on a list of equivalent questions and a list of equivalent answers
separated by \n\n. Use as fewer words from the original question and answer as possible.

The record:

questions: {questions}
answers: {answers}

Provide the output in parsable JSON without using code blocks with the following structure:

["question1", "question2", ..., "question5"]
"""

In [17]:
from src.client_modules.llms.openai.azure_openai import AzureOpenAIClient
from tqdm.auto import tqdm


azure_oai = AzureOpenAIClient()

ground_truth_questions = {}
for intent in tqdm(intents):
    prompt = prompt_template.format(questions='\n\n'.join(intent["patterns"]), 
                                    answers='\n\n'.join(intent["responses"]))
    response = azure_oai.chat([{"role": "user", "content": prompt}])
    json_response = response.choices[0].message.content
    ground_truth_questions[intent["id"]] = json_response

c:\Users\a.ramirez.lopez\AppData\Local\miniconda3\envs\cs-qa\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 533/533 [10:42<00:00,  1.20s/it]


In [33]:
import json

with open("../../data/raw_evaluation_ground_truth.json", "w") as file:
    json.dump(ground_truth_questions, file)

In [29]:
from json import JSONDecodeError


ground_truth_results = []
wrong_json_index = []
for index, (id, questions) in tqdm(enumerate(ground_truth_questions.items())):
    try:
        json_questions = json.loads(questions)
        for json_question in json_questions:
            ind_ground_truth = {"id": id,
                                "question": json_question}
            ground_truth_results.append(ind_ground_truth)
    except JSONDecodeError:
        wrong_json_index.append(index)

533it [00:00, 269085.70it/s]


In [35]:
import pandas as pd


pd.DataFrame(ground_truth_results).to_csv("../../data/evaluation_ground_truth.csv", index=False)